In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Imports

In [2]:
from fastai.text import *

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

## Functions

In [ ]:
def rounding_can_fuck_off(x):
    return np.floor(x) if ((x) - int(x)) < 0.5 else np.ceil(x)

## Config

In [4]:
data_path = Path('../data')
bs = 16

## Get data

### Read data in

In [ ]:
data_path.ls()

In [ ]:
attributes = pd.read_csv(data_path/'attributes.csv.zip', compression='zip', encoding='latin1')
train = pd.read_csv(data_path/'train.csv.zip', compression='zip', encoding='latin1')
test = pd.read_csv(data_path/'test.csv.zip', compression='zip', encoding='latin1')
product_desc = pd.read_csv(data_path/'product_descriptions.csv.zip', compression='zip', encoding='latin1')

### Inspect data

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
attributes.head()

In [ ]:
product_desc.head()

## Process data

In [ ]:
# # # Deal with N/As in attributes data (drop empty records and fill in name and values with empty string)
attr = attributes.dropna(how='all')
attr.loc[:, ['name','value']] = attr.loc[:, ['name','value']].fillna('')

# # # Ensure UID is int
attr.loc[:, 'product_uid'] = attr.loc[:, 'product_uid'].apply(lambda x: int(x))

# # # If "bullet" in attribute name then asserting name is meaningless - make an empty string
attr.loc[:, 'name'] = attr.loc[:, 'name'].apply(lambda x: '' if "Bullet" in x else x)

# # # Group name and value in attributes into single column, separated by a tab and ending in newline (for grouping stage next)
attr.loc[:, 'attributes'] = r' \ans ' + attr.loc[:, 'name'] + r' \ane \avs ' + attr.loc[:, 'value'] + r' \ave '

# # # Drop name and values, groupby UID and sum grouped values, reset index...
# # # ...(ie. all attributes in single cell now, separated by newlines as set up above)
attr = attr.drop(['name','value'], axis=1).groupby('product_uid').sum().reset_index()

In [ ]:
attr.head()

In [ ]:
# # # Merge all data into one master dataframe by merging descriptions and attributes onto training data on UID...
# # # ...Fill any NAs with empty string
data = pd.merge(train, product_desc, how='left', on='product_uid')\
.drop('id', axis=1)\
.merge(attr, on='product_uid', how='left')\
.fillna('')

# # # Finally create a master index column, which will be used to reference individual search terms
data = data.reset_index(drop=True).reset_index(drop=True)

In [ ]:
data.loc[:, 'relevance'] = data.loc[:, 'relevance'].apply(rounding_can_fuck_off).apply(int)

In [ ]:
data['text'] = \
' \ss ' + data.loc[:, 'search_term'] + ' \es ' +\
' \spt ' + data.loc[:, 'product_title'] + ' \ept ' +\
' \spd ' + data.loc[:, 'product_description'] + ' \epd ' +\
data.loc[:, 'attributes']

In [ ]:
final_data = data.drop(['product_title', 'search_term', 'product_description', 'attributes'], axis=1)

In [ ]:
# # # Merge all data into one master dataframe by merging descriptions and attributes onto training data on UID...
# # # ...Fill any NAs with empty string
data = pd.merge(test, product_desc, how='left', on='product_uid')\
.drop('id', axis=1)\
.merge(attr, on='product_uid', how='left')\
.fillna('')

# # # Finally create a master index column, which will be used to reference individual search terms
data = data.reset_index(drop=True).reset_index(drop=True)

In [ ]:
data['text'] = \
' \ss ' + data.loc[:, 'search_term'] + ' \es ' +\
' \spt ' + data.loc[:, 'product_title'] + ' \ept ' +\
' \spd ' + data.loc[:, 'product_description'] + ' \epd ' +\
data.loc[:, 'attributes']

In [ ]:
final_kaggle_data = data.drop(['product_title', 'search_term', 'product_description', 'attributes'], axis=1)

## Save

In [ ]:
final_data.to_csv(data_path/'train.csv')
final_kaggle_data.to_csv(data_path/'test.csv')

## Load

In [5]:
data = pd.read_csv(data_path/'train.csv').drop('Unnamed: 0', axis=1)
kaggle = pd.read_csv(data_path/'test.csv').drop('Unnamed: 0', axis=1)

In [6]:
texts = data['text'].append(kaggle['text'])

## Data Bunch

In [ ]:
# data_lm = (TextList.from_df(pd.DataFrame(texts))
#           .random_split_by_pct(0.1)
#           .label_for_lm()
#           .databunch(bs=bs))

In [ ]:
# data_lm.save(data_path/'data_lm.pkl')

In [7]:
data_lm = load_data(data_path, 'data_lm.pkl', bs=bs)

## Language Model

In [8]:
learn = language_model_learner(data_lm, Transformer, drop_mult=0.3)

In [ ]:
# learn.lr_find()

In [ ]:
# learn.recorder.plot(skip_end=15)

In [ ]:
# learn.fit_one_cycle(1, 3e-2, moms=(0.8,0.7))

In [ ]:
# learn.save('fit_head')

In [9]:
learn.load('fit_head')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (216684 items)
x: LMTextList
xxbos \ ss angle bracket \ es \ spt xxmaj simpson xxmaj strong - xxmaj tie 12-gauge xxmaj angle \ ept \ spd xxmaj not only do angles make joints stronger , they also provide more consistent , straight corners . xxmaj simpson xxmaj strong - xxmaj tie offers a wide variety of angles in various sizes and thicknesses to handle light - duty jobs or projects where a structural connection is needed . xxmaj some can be bent ( skewed ) to match the project . xxmaj for outdoor projects or those where moisture is present , use our xxup zmax zinc - coated connectors , which provide extra resistance against corrosion ( look for a " z " at the end of the model number).versatile connector for various 90 connections and home repair projectsstronger than angled nailing or screw fastening alonehelp ensure joints are consistently straight and strongdimensions : 3 in . x 3 in . x 1 - 1 / 2 in . xxmaj made from 12-gauge st

In [10]:
learn.unfreeze()

In [11]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy


KeyboardInterrupt: 

In [ ]:
learn.save('fine_tuned')